In [1]:
print("hlo")

hlo


In [1]:
dataset_path = r"/kaggle/input/forest-fire-dataset"
test_dir = r"/kaggle/input/forest-fire-dataset/test"
train_dir = r"/kaggle/input/forest-fire-dataset/train"
data_splitting_path = r"/kaggle/input/data-splitting/Data splitting"

In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.0 MB/s eta 0:00:0000:0100:01


In [2]:
import os
import random
import shutil
import glob # Use glob for robust file listing

# --- Configuration ---

# Path to the ORIGINAL dataset in /kaggle/input (read-only)
original_dataset_path = r"/kaggle/input/forest-fire-dataset"
original_train_images = os.path.join(original_dataset_path, "train", "images")
original_train_labels = os.path.join(original_dataset_path, "train", "labels")
original_test_images = os.path.join(original_dataset_path, "test", "images")
original_test_labels = os.path.join(original_dataset_path, "test", "labels")


# Path for the NEW working copy of the dataset in /kaggle/working (writable)
working_dataset_path = r"/kaggle/working/dfire_working_copy"
working_train_images = os.path.join(working_dataset_path, "train", "images")
working_train_labels = os.path.join(working_dataset_path, "train", "labels")
working_val_images = os.path.join(working_dataset_path, "val", "images")
working_val_labels = os.path.join(working_dataset_path, "val", "labels")
working_test_images = os.path.join(working_dataset_path, "test", "images")
working_test_labels = os.path.join(working_dataset_path, "test", "labels")


# Define the percentage of the *original* training data to use for validation
validation_split_percentage = 0.10 # 10% for validation

# Define the classes (based on your dataset exploration: 0=Fire, 1=Smoke)
class_names = ['Fire', 'Smoke']
num_classes = len(class_names)

# --- Step 1: Copy Original Data to Working Directory ---
print(f"Copying dataset from {original_dataset_path} to writable {working_dataset_path}")

# Create working directories
os.makedirs(working_train_images, exist_ok=True)
os.makedirs(working_train_labels, exist_ok=True)
os.makedirs(working_test_images, exist_ok=True)
os.makedirs(working_test_labels, exist_ok=True)
os.makedirs(working_val_images, exist_ok=True) # Create val now, will populate later
os.makedirs(working_val_labels, exist_ok=True) # Create val now, will populate later

# Copy Train Data
print("Copying train images and labels...")
train_image_files = glob.glob(os.path.join(original_train_images, '*.*'))
for img_src_path in train_image_files:
    if os.path.isfile(img_src_path):
        img_name = os.path.basename(img_src_path)
        label_name = os.path.splitext(img_name)[0] + ".txt"
        label_src_path = os.path.join(original_train_labels, label_name)

        img_dst_path = os.path.join(working_train_images, img_name)
        label_dst_path = os.path.join(working_train_labels, label_name)

        try:
            shutil.copy2(img_src_path, img_dst_path)
            if os.path.exists(label_src_path): # Copy label only if it exists
                 shutil.copy2(label_src_path, label_dst_path)
            # If label doesn't exist, it's a negative sample - that's fine, no label needed in working copy either
        except Exception as e:
             print(f"Error copying train file {img_name}: {e}")

# Copy Test Data
print("Copying test images and labels...")
test_image_files = glob.glob(os.path.join(original_test_images, '*.*'))
for img_src_path in test_image_files:
    if os.path.isfile(img_src_path):
        img_name = os.path.basename(img_src_path)
        label_name = os.path.splitext(img_name)[0] + ".txt"
        label_src_path = os.path.join(original_test_labels, label_name)

        img_dst_path = os.path.join(working_test_images, img_name)
        label_dst_path = os.path.join(working_test_labels, label_name)

        try:
            shutil.copy2(img_src_path, img_dst_path)
            if os.path.exists(label_src_path): # Copy label only if it exists
                 shutil.copy2(label_src_path, label_dst_path)
            # If label doesn't exist, it's a negative sample
        except Exception as e:
             print(f"Error copying test file {img_name}: {e}")

print("Finished copying data to working directory.")


# --- Step 2: Split Validation Set from the Working Training Data ---
print(f"\nSplitting {validation_split_percentage*100}% of data from working train set to working val set")

# Get the list of image files in the working train set
image_files = [os.path.basename(f) for f in glob.glob(os.path.join(working_train_images, '*.*')) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

if not image_files:
    print(f"Error: No image files found in working train folder {working_train_images}. Cannot split validation data.")
else:
    # Calculate the number of images to move to the validation set
    num_val_images = int(validation_split_percentage * len(image_files))

    if num_val_images == 0 and len(image_files) > 0:
         print(f"Info: Calculated 0 validation images. Will move at least 1 if total images > 0.")
         num_val_images = max(1, num_val_images) # Ensure at least one image if possible

    if num_val_images > len(image_files):
         print(f"Warning: Number of validation images to move ({num_val_images}) is greater than total working train images ({len(image_files)}). Moving all images.")
         num_val_images = len(image_files)

    print(f"Moving {num_val_images} images and labels to validation set...")

    # Randomly select the images to move
    try:
        val_image_files = random.sample(image_files, num_val_images)

        # Move the selected images and their corresponding labels to the validation set
        moved_count = 0
        skipped_label_count = 0
        for image_file in val_image_files:
            # Assume label filename is the same base name with .txt extension
            base_name = os.path.splitext(image_file)[0]
            label_file = base_name + ".txt"

            image_src = os.path.join(working_train_images, image_file)
            image_dst = os.path.join(working_val_images, image_file)

            label_src = os.path.join(working_train_labels, label_file)
            label_dst = os.path.join(working_val_labels, label_file)

            try:
               shutil.move(image_src, image_dst)
               if os.path.exists(label_src): # Only move label if it exists
                   shutil.move(label_src, label_dst)
               else:
                   # This image might be a negative sample (no label file) - that's fine
                   # print(f"Info: Label file not found for {image_file} at {label_src}. Moving image only.")
                   skipped_label_count += 1
               moved_count += 1
            except Exception as e:
                print(f"Error moving file {image_file} or its label: {e}")

        print(f"Finished splitting validation data. Moved {moved_count} images.")
        if skipped_label_count > 0:
             print(f"Info: {skipped_label_count} moved images did not have corresponding label files.")

    except ValueError as e:
        print(f"Error during random sampling: {e}. This might happen if the requested sample size ({num_val_images}) is larger than the population ({len(image_files)}).")
    except Exception as e:
        print(f"An unexpected error occurred during splitting: {e}")


# --- Step 3: Create data.yaml File ---
# Place data.yaml in the working directory for easy access during training
data_yaml_path = os.path.join('/kaggle/working/', 'data.yaml')

# The paths in data.yaml should be relative to the 'path' field, which is working_dataset_path
data_yaml_content = f"""
path: {working_dataset_path} # Path to the dataset root directory in /kaggle/working
train: train/images  # Relative path to train images from 'path'
val: val/images # Relative path to validation images from 'path'
test: test/images # Relative path to test images from 'path'

nc: {num_classes}  # Number of classes (Fire and Smoke)
names: {class_names} # Class names ['Fire', 'Smoke']. Order corresponds to IDs 0 and 1.
"""

print(f"\nCreating data.yaml file at: {data_yaml_path}")
try:
    with open(data_yaml_path, 'w') as f:
        f.write(data_yaml_content)
    print("data.yaml created successfully.")
    print("\nContent of data.yaml:")
    print(data_yaml_content)
except Exception as e:
    print(f"Error creating data.yaml: {e}")


print(f"Data copied to working directory {working_dataset_path}.")
print(f"Validation set split created within the working copy.")


Copying dataset from /kaggle/input/forest-fire-dataset to writable /kaggle/working/dfire_working_copy
Copying train images and labels...
Copying test images and labels...
Finished copying data to working directory.

Splitting 10.0% of data from working train set to working val set
Moving 1722 images and labels to validation set...
Finished splitting validation data. Moved 1722 images.

Creating data.yaml file at: /kaggle/working/data.yaml
data.yaml created successfully.

Content of data.yaml:

path: /kaggle/working/dfire_working_copy # Path to the dataset root directory in /kaggle/working
train: train/images  # Relative path to train images from 'path'
val: val/images # Relative path to validation images from 'path'
test: test/images # Relative path to test images from 'path'

nc: 2  # Number of classes (Fire and Smoke)
names: ['Fire', 'Smoke'] # Class names ['Fire', 'Smoke']. Order corresponds to IDs 0 and 1.

Data copied to working directory /kaggle/working/dfire_working_copy.
Valida

In [ ]:
from ultralytics import YOLO
import os # Import os to join path

# Path to your data.yaml file
data_yaml_file = '/kaggle/working/data.yaml'

# Load a model
model = YOLO('yolov8s.pt')  # load an official model

# Define project and run name
PROJECT = 'fire_detection' # Or any project name
NAME = 'yolov8s_dfire_split' # Name for this specific training run

model.train(
   data = 'data.yaml',
   task = 'detect',
   epochs = 100,
   verbose = True,
   batch = 64,
   imgsz = 640,
   patience = 20,
   save = True,
   device = 0,
   workers = 8,
   project = PROJECT,
   name = NAME,
   cos_lr = True,
   lr0 = 0.0001,
   lrf = 0.00001,
   warmup_epochs = 3,
   warmup_bias_lr = 0.000001,
   optimizer = 'Adam',
   seed = 42,
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 241MB/s]


Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=1e-05, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_dfire_split, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pret

100%|██████████| 755k/755k [00:00<00:00, 27.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     43/100      15.1G      1.052     0.6643      1.106         45        640: 100%|██████████| 243/243 [04:05<00:00,  1.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:09<00:00,  1.55it/s]

                   all       1722       2103      0.806      0.746      0.805       0.48



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     44/100      15.1G      1.047     0.6576        1.1        161        640:  63%|██████▎   | 154/243 [02:35<01:29,  1.01s/it]

In [ ]:
print("hi")

In [ ]:
lastpt44_model_path = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data/last.pt"
args_yaml = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data/args.yaml"
data_yaml_file = '/kaggle/working/data.yaml'
result_csv = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data/results (2).csv"
bestpt44_model_path = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data/best.pt"

In [6]:
import os

from ultralytics import YOLO

previous_run_output_dir = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data" # <<< VERIFY THIS PATH

lastpt44_model_path = os.path.join(previous_run_output_dir, "last.pt")
# args.yaml and best.pt are expected in the same directory and will be used by YOLOv8

# Path to your data.yaml file in your current working directory (/kaggle/working)
# Make sure you have a copy of this file in /kaggle/working for this session
data_yaml_file = '/kaggle/working/data.yaml' # <<< VERIFY THIS PATH is correct for THIS session



print(f"Loading model from checkpoint: {lastpt44_model_path}")
try:
    model = YOLO(lastpt44_model_path)
    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Checkpoint file not found at {lastpt44_model_path}. Please verify the path.")
    # You would typically stop execution here if the checkpoint is not found


# --- Step 2: Resume Training ---
print("\nResuming training...")


PROJECT = 'fire_detection' # Original project name
NAME = 'yolov8s_dfire_split' # Original run name


try:
    model.train(
       data = data_yaml_file, # Provide the data.yaml path for this session
       task = 'detect',
       epochs = 100,
       verbose = True,
       batch = 64,
       imgsz = 640,
       patience = 10,
       save = True,
       device = 0,
       workers = 8,
       project = PROJECT,
       name = NAME,
       cos_lr = True,
       lr0 = 0.0001,
       lrf = 0.00001,
       warmup_epochs = 3,
       warmup_bias_lr = 0.000001,
       optimizer = 'Adam',
       seed = 42,        # Usually loaded from args.yaml
    )
    print("\nTraining resumed successfully.")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")


Loading model from checkpoint: /kaggle/input/resuming-44th-epoch/44th_epoch_data/last.pt
Model loaded successfully.

Resuming training...
Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=1e-05, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/resuming-44th-epoch/44th_epoch_data/last.pt, momentum=0.

train: Scanning /kaggle/working/dfire_working_copy/train/labels.cache... 15499 images, 7025 backgrounds, 0 corrupt: 100%|██████████| 15499/15499 [00:00<?, ?it/s]

train: /kaggle/working/dfire_working_copy/train/images/WEB02521.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07199.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07271.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07278.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07297.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07305.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07312.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07534.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07535.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dfire_working_copy/train/images/WEB07536.jpg: corrupt JPEG restored 

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 346.3±181.0 MB/s, size: 208.3 KB)


val: Scanning /kaggle/working/dfire_working_copy/val/labels.cache... 1722 images, 808 backgrounds, 0 corrupt: 100%|██████████| 1722/1722 [00:00<?, ?it/s]

...


11 epochs completed in 0.781 hours.
Optimizer stripped from fire_detection/yolov8s_dfire_split3/weights/last.pt, 22.5MB
Optimizer stripped from fire_detection/yolov8s_dfire_split3/weights/best.pt, 22.5MB

Validating fire_detection/yolov8s_dfire_split3/weights/best.pt...
Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:11<00:00,  1.22it/s]


                   all       1722       2039      0.919      0.881      0.936      0.681
                  Fire        807        916      0.954      0.922       0.96      0.756
                 Smoke        457       1123      0.884      0.841      0.912      0.606


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 2.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to fire_detection/yolov8s_dfire_split3

Training resumed successfully.


In [ ]:
from ultralytics import YOLO
model = YOLO('project_name/experiment_name/weights/best.pt')
model.val(split='test', batch=48, imgsz=640, verbose=True, conf = 0.1, iou = 0.5)

In [3]:
import os

from ultralytics import YOLO

previous_run_output_dir = r"/kaggle/input/resuming-44th-epoch/44th_epoch_data" # <<< VERIFY THIS PATH

bestpt44_model_path = os.path.join(previous_run_output_dir, "best.pt")
# args.yaml and best.pt are expected in the same directory and will be used by YOLOv8

# Path to your data.yaml file in your current working directory (/kaggle/working)
# Make sure you have a copy of this file in /kaggle/working for this session
data_yaml_file = '/kaggle/working/data.yaml' # <<< VERIFY THIS PATH is correct for THIS session



print(f"Loading model from checkpoint: {bestpt44_model_path}")
try:
    model = YOLO(bestpt44_model_path)
    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Checkpoint file not found at {bestpt44_model_path}. Please verify the path.")
    # You would typically stop execution here if the checkpoint is not found


# --- Step 2: Resume Training ---
print("\nResuming training...")


PROJECT = 'fire_detection' # Original project name
NAME = 'yolov8s_dfire_split' # Original run name


try:
    model.train(
       data = data_yaml_file, # Provide the data.yaml path for this session
       task = 'detect',
       epochs = 100,
       verbose = True,
       batch = 64,
       imgsz = 640,
       patience = 10,
       save = True,
       device = 0,
       workers = 8,
       project = PROJECT,
       name = NAME,
       cos_lr = True,
       lr0 = 0.0001,
       lrf = 0.00001,
       warmup_epochs = 3,
       warmup_bias_lr = 0.000001,
       optimizer = 'Adam',
       seed = 42,        # Usually loaded from args.yaml
    )
    print("\nTraining resumed successfully.")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading model from checkpoint: /kaggle/input/resuming-44th-epoch/44th_epoch_data/best.pt
Model loaded successfully.

Resuming training...
Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, form

100%|██████████| 755k/755k [00:00<00:00, 20.5MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 5.35M/5.35M [00:00<00:00, 108MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2123.4±928.5 MB/s, size: 147.9 KB)


...


11 epochs completed in 0.781 hours.
Optimizer stripped from fire_detection/yolov8s_dfire_split/weights/last.pt, 22.5MB
Optimizer stripped from fire_detection/yolov8s_dfire_split/weights/best.pt, 22.5MB

Validating fire_detection/yolov8s_dfire_split/weights/best.pt...
Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:11<00:00,  1.24it/s]


                   all       1722       2275      0.907      0.854      0.923       0.65
                  Fire        887       1002      0.943      0.914      0.963      0.732
                 Smoke        498       1273      0.871      0.794      0.883      0.567


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 2.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to fire_detection/yolov8s_dfire_split

Training resumed successfully.


<function print(*args, sep=' ', end='\n', file=None, flush=False)>